In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import load_model


In [2]:
test = pd.read_csv("./dataset/test.csv")
test.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,SCHE4YSTDVPVZVXW,39.0,2016-01-13 07:45:08.000,205,Ivanka,No,Fashion,0,90,0,380.0,16.0,2576.0,3340
1,ACCEGCATKHNRXUHW,49.0,2015-08-23 20:37:05.000,3,Isaak,Yes,Fashion,0,87,0,393.0,16.0,1202.0,1955
2,NKCE6GJ5XVJDXNNZ,1.0,2015-11-14 18:12:39.000,183,Analiese,No,Technology,2,55,0,493.0,9.0,7175.0,15715
3,NKCEB8BK3ZXDHDHM,8.0,2015-11-21 04:56:19.000,358,Rusty,Yes,Child_care,3,86,0,303.0,16.0,5404.0,13078
4,TOPEFDXSAHRNPF94,33.0,2015-05-06 15:09:46.000,167,Eloise,No,Technology,0,27,0,567.0,16.0,4069.0,6244


In [3]:
data = test.copy()
data.isna().sum()

Product_id           0
Stall_no             1
instock_date         0
Market_Category      0
Customer_name       53
Loyalty_customer     0
Product_Category     0
Grade                0
Demand               0
Discount_avail       0
charges_1           36
charges_2 (%)        5
Minimum_price       14
Maximum_price        0
dtype: int64

In [4]:
range_val = data["Maximum_price"]-data["Minimum_price"]
range_val_mean = range_val.mean()
range_val_mean

2788.0863583138175

In [6]:
minimum_mean_price = []
for i,j in zip(data["Minimum_price"],data["Maximum_price"]):
    if i > 0:
        minimum_mean_price.append(i)
    else:
        minimum_mean_price.append(j-range_val_mean)

In [7]:
minimum_mean_price = pd.DataFrame(minimum_mean_price)
minimum_mean_price[1] = data.pop("Maximum_price")
minimum_mean_price.head()

,0,1
0,2576.0,3340
1,1202.0,1955
2,7175.0,15715
3,5404.0,13078
4,4069.0,6244


In [9]:
_ = data.pop("Minimum_price")

In [10]:
data = data.fillna(data.mean())
data.isna().sum()

Product_id           0
Stall_no             0
instock_date         0
Market_Category      0
Customer_name       53
Loyalty_customer     0
Product_Category     0
Grade                0
Demand               0
Discount_avail       0
charges_1            0
charges_2 (%)        0
dtype: int64

In [11]:
# data = data.replace(np.nan," ")
_ = data.pop("Customer_name")
data.isna().sum()

Product_id          0
Stall_no            0
instock_date        0
Market_Category     0
Loyalty_customer    0
Product_Category    0
Grade               0
Demand              0
Discount_avail      0
charges_1           0
charges_2 (%)       0
dtype: int64

In [12]:
product_id = data.pop("Product_id")

In [13]:
data["mean"] = (minimum_mean_price[0]+minimum_mean_price[1])/2
data.head()

,Stall_no,instock_date,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),mean
0,39.0,2016-01-13 07:45:08.000,205,No,Fashion,0,90,0,380.0,16.0,2958.0
1,49.0,2015-08-23 20:37:05.000,3,Yes,Fashion,0,87,0,393.0,16.0,1578.5
2,1.0,2015-11-14 18:12:39.000,183,No,Technology,2,55,0,493.0,9.0,11445.0
3,8.0,2015-11-21 04:56:19.000,358,Yes,Child_care,3,86,0,303.0,16.0,9241.0
4,33.0,2015-05-06 15:09:46.000,167,No,Technology,0,27,0,567.0,16.0,5156.5


In [14]:
product_cat = data.pop("Product_Category")

In [15]:
Grade = data.pop("Grade")

In [16]:
encoder = preprocessing.OrdinalEncoder()


In [17]:
product_cat = encoder.fit_transform(np.expand_dims(product_cat,1))
product_cat

array([[3.],
       [3.],
       [9.],
       ...,
       [2.],
       [0.],
       [8.]])

In [18]:
data = encoder.fit_transform(data)

In [19]:
Grade = encoder.fit_transform(np.expand_dims(Grade,1))

In [20]:
data.shape

(3430, 9)

In [21]:
onehot = preprocessing.OneHotEncoder(sparse=False)

In [22]:
product_cat = onehot.fit_transform(product_cat)
product_cat.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(3430, 10)

In [24]:
Grade = onehot.fit_transform(Grade)
Grade.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(3430, 4)

In [321]:
# data = preprocessing.normalize(data)
# data

In [25]:
data = preprocessing.minmax_scale(data)
data

array([[0.78      , 0.71534733, 0.6367713 , ..., 0.61777778, 0.94117647,
        0.24919303],
       [0.98      , 0.30726979, 0.01345291, ..., 0.64666667, 0.94117647,
        0.06552615],
       [0.        , 0.55476575, 0.56502242, ..., 0.73111111, 0.47058824,
        0.97127179],
       ...,
       [0.26      , 0.91017771, 0.3632287 , ..., 0.11111111, 1.        ,
        0.69657844],
       [0.        , 0.13537964, 0.35426009, ..., 0.5       , 0.88235294,
        0.31052292],
       [0.04      , 0.1812601 , 0.52914798, ..., 0.28444444, 0.47058824,
        0.76371853]])

In [26]:
final = pd.concat([pd.DataFrame(data),pd.DataFrame(product_cat),pd.DataFrame(Grade)],axis=1)
final.head()

,0,1,2,3,4,5,6,7,8,0,...,4,5,6,7,8,9,0,1,2,3
0,0.78,0.715347,0.636771,0.0,0.898990,0.0,0.617778,0.941176,0.249193,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.98,0.307270,0.013453,1.0,0.868687,0.0,0.646667,0.941176,0.065526,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.00,0.554766,0.565022,0.0,0.545455,0.0,0.731111,0.470588,0.971272,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.14,0.621325,0.816143,1.0,0.858586,0.0,0.453333,0.941176,0.924467,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.66,0.053958,0.538117,0.0,0.262626,0.0,0.855556,0.941176,0.612008,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [27]:
final = np.array(final)
final.shape

(3430, 23)

In [191]:
model = load_model("Best_model_2759")

In [192]:
result = model.predict(final)

In [193]:
result

array([[ 3222.652 ],
       [ 2162.6323],
       [12025.029 ],
       ...,
       [ 5979.504 ],
       [ 3536.1746],
       [ 1115.9718]], dtype=float32)

In [194]:
count = 0
for i in range(len(result)):
    if minimum_mean_price[0][i]<=result[i]<=minimum_mean_price[1][i]:
        count+=1
count

2759

In [195]:
submission = pd.DataFrame(product_id)

In [196]:
submission["Selling_Price"] = result
submission.head()

,Product_id,Selling_Price
0,SCHE4YSTDVPVZVXW,3222.652100
1,ACCEGCATKHNRXUHW,2162.632324
2,NKCE6GJ5XVJDXNNZ,12025.029297
3,NKCEB8BK3ZXDHDHM,9855.031250
4,TOPEFDXSAHRNPF94,5677.274414


In [197]:
submission.to_csv("submission.csv",index=False)